# Python OOP for Ops Tutorial

This tutorial is to demonstrate the basic principles of Object-Oriented Programming in Python, using an Operations-like scenario. This knowledge will help you take your Operations programming skills to the next level, from basic single-purpose scripts to reusable, flexible code.

The core scenario is: you have several remote routers and your task is to determine which of these are busiest. To enable this, you decide to compile throughput metrics on the interfaces of each router as a proxy for "busy-ness".


## Scenario 1

At first, all routers are the same. A single, simple script iterates over all routers' addresses, logs in to each and pulls its traffic data. (To focus on the core lesson, the login credentials are the same for all routers. Of course, in real life, no-one would do that these days, right?)

## Scenario 2

Owing to security concerns, a later version of firmware has been rolled out that changes the API. For Reasons, the rollout of the new firmware could not be completed across the board: some routers have been upgraded, some haven't.
Owing to security concerns, a later version of firmware has been rolled out that changes the API. For Reasons, the rollout of the new firmware could not be completed across the board: some routers have been upgraded, some haven't.

### Sequence of events:

* New TutorRouter firmware is rolled out to some routers. The script can no longer log in to or get traffic data from _all_ the routers in the same way. Modify script
    with if/else.
    * Observe that further firmware updates will cause headaches...
    * Observe that other scripts doing different tasks will have to be modified.

### Demonstrate

* **Inheritance** - DRYness.
  * Each device _is a_ `TutorRouter` with some commonalities, e.g. IP address; health check path is always the same.
  * All TutorRouters can output total ingress and total egress.
  * Different login methodologies.
* **Encapsulation** - contain attributes (properties and methods) together in Objects. Each device _has_...
  * **Properties**: IP address/Hostname, set of interfaces.
  * **Methods**: a _method_ of logging in, a _method_ for checking health, a _method_ of pulling interface stats.
* **Abstraction** - hide implementation details that users of your class don't need or want to deal with.
  * Hide the details of how login is achieved.
  * Return the data (e.g. interface throughput) in a format that's easy to consume.
* **Polymorphism** - allow all routers to be treated the same way transparently.
  * Although different "shapes", they are all routers with the same functions.
  * No special logic should be needed by the user for routers with different firmware.

### Set up the simulation

In [1]:
# Load the libraries
import os, sys
sys.path.insert(0, "../src")
from tutor_router.scenarios import Scenario, NotFoundError

# Set up the scenario
scenario = Scenario.scenario_1()

# Make these look a bit like `requests` calls.
request = scenario.request
get = scenario.get
post = scenario.post

# Get a list of host names, for convenience.
hosts = tuple(scenario.hosts())

These are the hosts. We can use these as hostnames.

In [2]:
hosts

('TutorRouter-1',
 'TutorRouter-2',
 'TutorRouter-3',
 'TutorRouter-4',
 'TutorRouter-5',
 'TutorRouter-6',
 'TutorRouter-7',
 'TutorRouter-8',
 'TutorRouter-9',
 'TutorRouter-10',
 'TutorRouter-11',
 'TutorRouter-12',
 'TutorRouter-13',
 'TutorRouter-14',
 'TutorRouter-15',
 'TutorRouter-16',
 'TutorRouter-17',
 'TutorRouter-18',
 'TutorRouter-19',
 'TutorRouter-20')

## Building a typical Ops script

**NOTE** keep in mind that this is a _simulation_. It is _somewhat_ like what you'd do with `requests` against real hosts, but not _exactly_ so. In order to concentrate on the core lessons, many details have been left out , particularly around how you'd acquire and use authentication tokens.

There's a healthcheck endpoint on each router. Let's poll them all first.

In [3]:
# Iterate over all routers
for hostname in scenario.hosts():

    # Perform a health check on the current router.
    health = scenario.get(hostname, "healthcheck")
    print(health)

(200, '{"host": "TutorRouter-1", "version": 1, "health": "good"}')
(200, '{"host": "TutorRouter-2", "version": 1, "health": "good"}')
(200, '{"host": "TutorRouter-3", "version": 1, "health": "good"}')
(200, '{"host": "TutorRouter-4", "version": 1, "health": "good"}')
(200, '{"host": "TutorRouter-5", "version": 1, "health": "good"}')
(200, '{"host": "TutorRouter-6", "version": 1, "health": "good"}')
(200, '{"host": "TutorRouter-7", "version": 1, "health": "good"}')


NotFoundError: Connection error

Bummer, some of them are down. Never mind, we can catch that exception and ignore those routers until they are fixed.

In [4]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

Host TutorRouter-1 is up
Host TutorRouter-2 is up
Host TutorRouter-3 is up
Host TutorRouter-4 is up
Host TutorRouter-5 is up
Host TutorRouter-6 is up
Host TutorRouter-7 is up
Host TutorRouter-8 is not available
Host TutorRouter-9 is up
Host TutorRouter-10 is up
Host TutorRouter-11 is not available
Host TutorRouter-12 is up
Host TutorRouter-13 is up
Host TutorRouter-14 is not available
Host TutorRouter-15 is not available
Host TutorRouter-16 is up
Host TutorRouter-17 is up
Host TutorRouter-18 is up
Host TutorRouter-19 is up
Host TutorRouter-20 is not available


Cool, now we can get the interface stats.

In [5]:
# Iterate over all routers
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # Now get the interface stats.
    status_code, response = get(hostname, "interfaces")
    print(f"  interfaces: ({status_code}) {response}")

Host TutorRouter-1 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-2 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-3 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-4 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-5 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-6 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-7 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-8 is not available
Host TutorRouter-9 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-10 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-11 is not available
Host TutorRouter-12 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-13 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-14 is not available
Host TutorRouter-15 is not available
Host TutorRouter-16 is up
  interfaces: (403) {"message": "Forbidden"}
Host Tuto

Ah, that function needs login. OK let's do that.

In [6]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    print(f"  log in: ({status_code}) {response}")

Host TutorRouter-1 is up
  log in: (200) faca9e13-8f6f-4923-9541-d73fb721247a
Host TutorRouter-2 is up
  log in: (200) 3a14259c-3c95-4d44-b075-99dc79bc8cb6
Host TutorRouter-3 is up
  log in: (200) 8e387407-a59b-4c45-b829-c1fd28a6a2c4
Host TutorRouter-4 is up
  log in: (200) c1dc5710-a8d9-4a7f-90d5-a0b90855f756
Host TutorRouter-5 is up
  log in: (200) d70b3a6a-cea3-4721-99f6-3708e625b910
Host TutorRouter-6 is up
  log in: (200) 9158b89f-2735-4904-84b6-6bfea70fad53
Host TutorRouter-7 is up
  log in: (200) 037f4b9b-ec7b-441d-8b9e-b2abb316504f
Host TutorRouter-8 is not available
Host TutorRouter-9 is up
  log in: (200) b0d5de7a-6962-4b5e-acf6-523f8751e7d8
Host TutorRouter-10 is up
  log in: (200) 94c661e7-0a30-45c0-a2ee-73b8048482d1
Host TutorRouter-11 is not available
Host TutorRouter-12 is up
  log in: (200) b788ae0c-0b5f-4865-ab9b-52a95d5aaf74
Host TutorRouter-13 is up
  log in: (200) 7d6e0bdb-ba40-43f2-91f2-7a08368b3592
Host TutorRouter-14 is not available
Host TutorRouter-15 is not av

The response to login (again: in this simulation) is an auth token that we have to pass in the headers with every request to a protected endpoint.

Let's do that.

Also, let's not print out the auth token any more :-)

In [7]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    print(f"  Logged in")
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Token": auth_token})
    if status_code != 200:
        print(f"  Failed to get interface stats: {response}")
        continue
    print(f"  Interfaces: ({status_code}) {response}")

Host TutorRouter-1 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 827712, "down": 754591}, "mgmt": {"up": 735201, "down": 697089}}
Host TutorRouter-2 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 785745, "down": 385503}, "mgmt": {"up": 435444, "down": 457297}}
Host TutorRouter-3 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 340029, "down": 122586}, "mgmt": {"up": 919619, "down": 476388}}
Host TutorRouter-4 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 179571, "down": 177236}, "mgmt": {"up": 631625, "down": 125410}}
Host TutorRouter-5 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 509555, "down": 694249}, "mgmt": {"up": 939222, "down": 29907}}
Host TutorRouter-6 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 963076, "down": 583812}, "mgmt": {"up": 11015, "down": 758043}}
Host TutorRouter-7 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 553695, "down": 822265}, "mgmt": {"up": 534772, "down": 326094}}
Host TutorRouter-8 is not available


Noice! Now we can just add the "up" and "down" for `eth0`...

In [8]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    print(f"  Logged in")
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Token": auth_token})
    if status_code != 200:
        print(f"  Failed to get interface stats")
        continue
    total_throughput = response["eth0"]["up"] + response["eth0"]["down"]
    print(f"  Throughput: {total_throughput} bits per second")

Host TutorRouter-1 is up
  Logged in


TypeError: string indices must be integers

What the heck does that mean?

Ah. Turns out the response isn't a Python object, it's a JSON string blob. Let's fix that.

In [9]:
import json

# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Token": auth_token})
    data = json.loads(response)
    total_throughput = data["eth0"]["up"] + data["eth0"]["down"]
    print(f"  Throughput: {total_throughput} bits per second")

Host TutorRouter-1 is up
  Throughput: 1582303 bits per second
Host TutorRouter-2 is up
  Throughput: 1171248 bits per second
Host TutorRouter-3 is up
  Throughput: 462615 bits per second
Host TutorRouter-4 is up
  Throughput: 356807 bits per second
Host TutorRouter-5 is up
  Throughput: 1203804 bits per second
Host TutorRouter-6 is up
  Throughput: 1546888 bits per second
Host TutorRouter-7 is up
  Throughput: 1375960 bits per second
Host TutorRouter-8 is not available
Host TutorRouter-9 is up
  Throughput: 1004612 bits per second
Host TutorRouter-10 is up
  Throughput: 999571 bits per second
Host TutorRouter-11 is not available
Host TutorRouter-12 is up
  Throughput: 540614 bits per second
Host TutorRouter-13 is up
  Throughput: 1723701 bits per second
Host TutorRouter-14 is not available
Host TutorRouter-15 is not available
Host TutorRouter-16 is up
  Throughput: 122488 bits per second
Host TutorRouter-17 is up
  Throughput: 636715 bits per second
Host TutorRouter-18 is up
  Through

Job done!

But wait...